In [277]:
from json.decoder import JSONDecodeError

In [263]:
json.loads('{"Media": ["prąd", "kanalizacja", "woda"], "Dojazd": ["asfaltowy"]}')

{'Media': ['prąd', 'kanalizacja', 'woda'], 'Dojazd': ['asfaltowy']}

In [278]:
from dataclasses import dataclass
from datetime import datetime


@dataclass
class OtodomOffer:
    # TODO: inherit from data/models/common.py Offer(). Put there to_dict()
    number_id: int
    short_id: str
    long_id: str
    url: str
    title: str
    price: int
    advertiser_type: str
    advert_type: str
    utc_created_at: datetime
    utc_scraped_at: datetime
    description: str
    city: str
    subregion: str
    province: str
    location: str
    latitude: float
    longitude: float

    def to_dict(self):
        output_dict = {}
        for key, value in self.__dict__.items():
            try:
                value = json.loads(value)
            except TypeError:
                value = value
            except JSONDecodeError:
                value = value
                
            output_dict[key] = value
        
        return output_dict

@dataclass
class OtodomLotOffer(OtodomOffer):
    lot_area: int
    lot_features: str
    vicinity: str


@dataclass
class OtodomHouseOffer(OtodomOffer):
    market: str
    building_type: str
    house_features: str
    lot_area: int
    house_area: int
    n_rooms: int
    floors_info: str
    heating: str
    build_year: int
    media: str
    vicinity: str


@dataclass
class OtodomApartmentOffer(OtodomOffer):
    market: str
    status: list[str]
    apartment_features: str
    apartment_area: int
    build_year: int
    building_floors_num: int
    building_type: str
    elevator: bool
    media: str
    heating: str
    rent: int
    n_rooms: int


In [2]:
import requests
from bs4 import BeautifulSoup

headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/113.0",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
"Accept-Language": "en-US,en;q=0.5",
"Accept-Encoding": "gzip, deflate, br",
"Connection": "keep-alive",
"DNT": "1",
"Sec-GPC": "1",
"TE": "trailers"}

In [119]:
url = "https://www.otodom.pl/pl/oferta/dzialka-przy-lesie-z-wz-puszcza-zielonka-ID4m1iL"
url1 = "https://www.otodom.pl/pl/oferta/dzialka-4-750-m-skarzysko-kamienna-ID4lW6h"

In [120]:
resp = requests.get(url1, headers=headers)
resp

<Response [200]>

In [121]:
offer_soup = BeautifulSoup(resp.text, "html.parser")

In [205]:
offer_data = offer_soup1.find("script", {"id": "__NEXT_DATA__"}).text
offer_json = json.loads(offer_data)
offer_json = offer_json["props"]["pageProps"]["ad"]

In [213]:
offer_json

{'id': 64329729,
 'market': '',
 'services': [],
 'publicId': '4lW6h',
 'slug': 'dzialka-4-750-m-skarzysko-kamienna-ID4lW6h',
 'advertiserType': 'business',
 'advertType': 'AGENCY',
 'source': 'urn:site:local',
 'createdAt': '2023-06-16T17:21:15+02:00',
 'modifiedAt': '2023-06-16T17:21:28+02:00',
 'description': '<p>Oferujemy do sprzedaży   działkę  inwestycyjną  usytuowaną  w miejscowości  Skarżysko Kam. liczącym około 50.000  mieszkańców.<br/> Skarżysko Kam.  to urokliwa miejscowość  o bogato rozwiniętej infrastrukturze,   położona w   północnej części województwa świętokrzyskiego .<br/> W  odległości około   37 km od Kielc  i około 140 km od Warszawy.  <br/>Miasto jest położone na przecięciu ważnych szlaków komunikacyjnych Warszawa  - Kraków oraz Łódź - Rzeszów.<br/>Przez miasto przechodzą  dwie drogi krajowe i jedna ekspresowa.<br/>Pobliski teren  to znakomite miejsce niedzielnych spacerów, wycieczek rowerowych i wędrówek pieszych po turystycznych szlakach  a także sportów wodnych.

In [227]:
from contextlib import suppress

In [232]:
.get()

1

In [236]:
country = offer_json["target"]["Country"]  # Polska
target = offer_json["target"]["OfferType"] # sprzedaz


number_id = offer_json["id"]
short_id = offer_json["publicId"]
long_id = offer_json["slug"]
url = offer_json["url"]
title = offer_json["title"]
price = offer_json["target"]["Price"]
advertiser_type = offer_json["advertiserType"]
advert_type = offer_json["advertType"]
utc_created_at = datetime.fromisoformat(offer_json["createdAt"]).replace(tzinfo=None)
utc_scraped_at = datetime.now(tz=timezone.utc)
description = unicodedata.normalize("NFKC", BeautifulSoup(offer_json["description"], "html.parser").text)
city = offer_json["location"]["address"]["city"]["name"]
subregion = offer_json["location"]["address"]["county"]["code"]
province = offer_json["location"]["address"]["province"]["code"]
location = "|".join(offer_json["target"].get("Location", []))
latitude = offer_json["location"]["coordinates"]["latitude"]
longitude = offer_json["location"]["coordinates"]["longitude"]
lot_area = int(float(offer_json["target"]["Area"]))
lot_features = json.dumps({category["label"]: category["values"] for category in offer_json.get("featuresByCategory")}, ensure_ascii=False)
vicinity = "|".join(offer_json["target"].get("Vicinity_types", []))

In [273]:
offer_model = OtodomLotOffer(
    number_id=number_id,
    short_id=short_id,
    long_id=long_id,
    url=url,
    title=title,
    price=price,
    advertiser_type=advertiser_type,
    advert_type=advert_type,
    utc_created_at=utc_created_at,
    utc_scraped_at=utc_scraped_at,
    description=description,
    city=city,
    subregion=subregion,
    province=province,
    location=location,
    longitude=longitude,
    latitude=latitude,
    lot_area=lot_area,
    lot_features=lot_features,
    vicinity=vicinity
)

offer_model

OtodomLotOffer(number_id=64329729, short_id='4lW6h', long_id='dzialka-4-750-m-skarzysko-kamienna-ID4lW6h', url='https://www.otodom.pl/pl/oferta/dzialka-4-750-m-skarzysko-kamienna-ID4lW6h', title='Działka, 4 750 m², Skarżysko-Kamienna', price=712000, advertiser_type='business', advert_type='AGENCY', utc_created_at=datetime.datetime(2023, 6, 16, 17, 21, 15), utc_scraped_at=datetime.datetime(2023, 6, 24, 10, 26, 50, 472996, tzinfo=datetime.timezone.utc), description='Oferujemy do sprzedaży   działkę  inwestycyjną  usytuowaną  w miejscowości  Skarżysko Kam. liczącym około 50.000  mieszkańców. Skarżysko Kam.  to urokliwa miejscowość  o bogato rozwiniętej infrastrukturze,   położona w   północnej części województwa świętokrzyskiego . W  odległości około   37 km od Kielc  i około 140 km od Warszawy.  Miasto jest położone na przecięciu ważnych szlaków komunikacyjnych Warszawa  - Kraków oraz Łódź - Rzeszów.Przez miasto przechodzą  dwie drogi krajowe i jedna ekspresowa.Pobliski teren  to znakomi

In [279]:
offer_model.to_dict()

{'number_id': 64329729,
 'short_id': '4lW6h',
 'long_id': 'dzialka-4-750-m-skarzysko-kamienna-ID4lW6h',
 'url': 'https://www.otodom.pl/pl/oferta/dzialka-4-750-m-skarzysko-kamienna-ID4lW6h',
 'title': 'Działka, 4 750 m², Skarżysko-Kamienna',
 'price': 712000,
 'advertiser_type': 'business',
 'advert_type': 'AGENCY',
 'utc_created_at': datetime.datetime(2023, 6, 16, 17, 21, 15),
 'utc_scraped_at': datetime.datetime(2023, 6, 24, 10, 26, 50, 472996, tzinfo=datetime.timezone.utc),
 'description': 'Oferujemy do sprzedaży   działkę  inwestycyjną  usytuowaną  w miejscowości  Skarżysko Kam. liczącym około 50.000  mieszkańców. Skarżysko Kam.  to urokliwa miejscowość  o bogato rozwiniętej infrastrukturze,   położona w   północnej części województwa świętokrzyskiego . W  odległości około   37 km od Kielc  i około 140 km od Warszawy.  Miasto jest położone na przecięciu ważnych szlaków komunikacyjnych Warszawa  - Kraków oraz Łódź - Rzeszów.Przez miasto przechodzą  dwie drogi krajowe i jedna ekspreso